# ForwardDiff.jl

*Student: Jarrett Revels*

*Mentors: Miles Lubin, Theodore Papamarkou*

*Sponsored by: Julia Summer of Code (Gordon and Betty Moore Foundation)*

## What is ForwardDiff.jl?

ForwardDiff.jl is a package that provides a type-based implementation of forward mode AD in the Julia language.

## What can I do with it?

Take derivatives, gradients, Jacobians, Hessians, and "Tensors" ($3^{\text{rd}}$ order derivatives) of native Julia functions (or any callable object, really).

## What does it look like? 


For the sake of example, let's use ForwardDiff.jl to take derivatives of a toy function $f:\mathbb{R}^n \rightarrow \mathbb{R}$:

$$ f(\vec{x}) = \sum_{i=1}^n \sin(x_i) + \prod_{i=1}^n \tan(x_i) \cdot \sum_{i=1}^n \sqrt{x_i} $$

In the Julia REPL:

```julia
julia> using ForwardDiff

julia> f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);

julia> x = rand(5) # small size for example's sake
5-element Array{Float64,1}:
 0.986403
 0.140913
 0.294963
 0.837125
 0.650451

julia> g = ForwardDiff.gradient(f); # g = ∇f

julia> g(x)
5-element Array{Float64,1}:
 1.01358
 2.50014
 1.72574
 1.10139
 1.2445

julia> j = ForwardDiff.jacobian(g); # j = J(∇f)

julia> j(x)
5x5 Array{Float64,2}:
 0.585111  3.48083  1.7706    0.994057  1.03257
 3.48083   1.06079  5.79299   3.25245   3.37871
 1.7706    5.79299  0.423981  1.65416   1.71818
 0.994057  3.25245  1.65416   0.251396  0.964566
 1.03257   3.37871  1.71818   0.964566  0.140689

julia> ForwardDiff.hessian(f, x) # H(f)(x) == J(∇f)(x), as expected 
5x5 Array{Float64,2}:
 0.585111  3.48083  1.7706    0.994057  1.03257
 3.48083   1.06079  5.79299   3.25245   3.37871
 1.7706    5.79299  0.423981  1.65416   1.71818
 0.994057  3.25245  1.65416   0.251396  0.964566
 1.03257   3.37871  1.71818   0.964566  0.140689
 ```

## Why Julia?

Unlike many other languages, **Julia's type-based operator overloading is fast and natural**, and one of the central design tenets of the langauge.

This is because Julia is a dynamic, JIT-compiled language - **the compiled bytecode of a Julia function is tied directly to the types with which the function is called**, so the compiler can optimize every Julia method for the specific input type at runtime.

For example:

```julia
julia> f(x) = sqrt(x)
f (generic function with 1 method)

julia> f(2) # method f(::Int64) is JIT-compiled and called
1.4142135623730951

julia> f(1+2im) # new method f(::Complex{Int64}) is JIT-compiled and called
1.272019649514069 + 0.7861513777574233im

julia> f(x::AbstractString) = "√$x" # if called on some sort of string, do this instead
f (generic function with 2 methods)

julia> f("2") # new method f(::ASCIIString) is JIT-compiled and called
"√2"

julia> f(2) # call the version of f(::Int64) we already compiled
1.4142135623730951
```

This strategy, referred to in Julia nomenclature as **multiple dispatch**, allows us to easily and efficiently overload core Julia methods (e.g. `sin`, `lgamma`, `log`, etc.)  to accumulate derivative information. 



## Types
ForwardDiff.jl overloads mathematical methods on three different types:

---

The **`GradientNumber{N}`** type represents a **generalized dual number** with $N$ $\epsilon$-components: 
$$g_N = a + \sum_{i=1}^N b_i \epsilon_i$$
Overloaded functions on vectors of `GradientNumbers` can be structured to behave like: 
$$f(\vec{x}_{g_N}) = f(\vec{x}) + \sum_{i=1}^N \frac{\delta f(\vec{x})}{\delta x_i} \epsilon_i$$

---

The **`HessianNumber{N}`** type represents a **hyper-dual number** whose dimensions scale with $N$:
$$h_N = a + \sum_{i=1}^N b_i \epsilon_i + \sum_{i,j=1}^N c_{ij} \epsilon_i \epsilon_j$$
Overloaded functions on vectors of `HessianNumbers` can be structured to behave like:
$$f(\vec{x}_{h_N}) = f(\vec{x}_{g_N}) + \sum_{i,j=1}^N \frac{\delta^2 f(\vec{x})}{\delta x_i \delta x_j} \epsilon_i \epsilon_j$$

---

The **`TensorNumber{N}`** type represents a **$3^{\text{rd}}$ order hyper-dual number** whose dimensions scale with $N$:
$$t_N = a + \sum_{i=1}^N b_i \epsilon_i + \sum_{i,j=1}^N c_{ij} \epsilon_i \epsilon_j + \sum_{i,j,k=1}^N d_{ijk} \epsilon_i \epsilon_j \epsilon_k$$
Overloaded functions on vectors of `TensorNumbers` can be structured to behave like:
$$f(\vec{x}_{t_N}) = f(\vec{x}_{h_N}) + \sum_{i,j,k=1}^N \frac{\delta^3 f(\vec{x})}{\delta x_i \delta x_j \delta x_k} \epsilon_i \epsilon_j \epsilon_k$$

## Vector-mode vs. Chunk-mode

ForwardDiff.jl can be configured to use two different strategies for derivative evaluation.

### vector-mode (default)
Evaluate all partial derivatives of $f(\vec{x})$ by attaching a unique $\epsilon_i$ to each $x_i$. For example, taking the gradient of $f:\mathbb{R}^N \to \mathbb{R}$ in vector-mode:

$$
\vec{x} = 
\begin{bmatrix}
x_1 \\
\vdots \\
x_i \\
\vdots \\
x_N
\end{bmatrix} \to
\vec{x}_{g_N} =
\begin{bmatrix}
x_1 + \epsilon_1 \\
\vdots \\
x_i + \epsilon_i \\
\vdots \\
x_N + \epsilon_N
\end{bmatrix} \to
f(\vec{x}_{g_N}) =
f(\vec{x}) + \sum_{i=1}^N \frac{\delta f(\vec{x})}{\delta x_i} \epsilon_i
$$

Properties: 

- `N` == length($\vec{x}$)
- Partials Storage Type: Julia's heap-allocated `Vectors`$^*$.
- Tradeoff: Only requires a single evaluation of $f$ to calculate any kind of derivative, but that single evaluation can require a lot of memory.

$*_{\text{Tuples are used in vector-mode when length(x) < 10, since there is little chance of "clogging" the stack at such low input dimensions.} }$

### chunk-mode
Evaluate partial derivatives of $f(\vec{x})$ in "chunks" of size $N$. For example, taking the gradient of $f:\mathbb{R}^4 \to \mathbb{R}$ in chunk-mode with $N=2$:

$$
\vec{x}_{g_{1,2}} = 
\begin{bmatrix}
x_1 + \epsilon_1 \\
x_2 + \epsilon_2 \\
x_3 \\
x_4
\end{bmatrix} \to
f(\vec{x}_{g_{1,2}}) = f(\vec{x}) + \frac{\delta f(\vec{x})}{\delta x_1} \epsilon_1 + \frac{\delta f(\vec{x})}{\delta x_2} \epsilon_2
$$

$$
\vec{x}_{g_{3,4}} = 
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 + \epsilon_1\\
x_4 + \epsilon_2
\end{bmatrix} \to
f(\vec{x}_{g_{3,4}}) = f(\vec{x}) + \frac{\delta f(\vec{x})}{\delta x_3} \epsilon_1 + \frac{\delta f(\vec{x})}{\delta x_4} \epsilon_2
$$


- `N` <= length($\vec{x}$)
- Tradeoff: Multiple $f$ evaluations are required, but less memory is required per evaluation
- Partials Storage Type: Julia's stack-allocated `Tuples` (fast member access with less GC overhead, but high dimensions can "clog" the stack).
- Number of evaluations of $f$ to compute $(k = \text{length}(x))$... 
    - $\nabla f(\vec{x})$: $\frac{k}{N}$
    - $J(f(\vec{x}))$: $\frac{k}{N}$
    - $H(f(\vec{x}))$: $\frac{k}{N} (k + 1 - \frac{1}{N}) \to O(\frac{k^2}{N})$
    - $T(f(\vec{x}))$: N/A (chunk-mode not supported for Tensors)

## Benchmarks

Every time shown below is the minimum of at least 5 trials.

All ForwardDiff evaluations are performed in vector-mode unless it is specified otherwise.

### Ackley Function

#### Definition

$$f(\vec{x}) = -a \exp( -b \sqrt{\frac{1}{k} \sum_{i=1}^k x^{2}_{i}}) - \exp(\frac{1}{k} \sum_{i=1}^k \cos(cx_{i})) + a + \exp(1)$$

#### Julia Code

```julia
function ackley(x)
    a, b, c = 20.0, -0.2, 2.0*π
    len_recip = inv(length(x))
    sum_sqrs = zero(eltype(x))
    sum_cos = sum_sqrs
    for i in x
        sum_cos += cos(c*i)
        sum_sqrs += i*i
    end
    return (-a * exp(b * sqrt(len_recip*sum_sqrs)) -
            exp(len_recip*sum_cos) + a + e)
end
```

#### Python Code (with AlgoPy)

```python
def ackley(x):
    a, b, c = 20.0, -0.2, 2.0*numpy.pi
    len_recip = 1.0/len(x)
    sum_sqrs, sum_cos = 0.0, 0.0
    for i in x:
        sum_cos += algopy.cos(c*i)
        sum_sqrs += i*i
    return (-a * algopy.exp(b*algopy.sqrt(len_recip*sum_sqrs)) -
            algopy.exp(len_recip*sum_cos) + a + numpy.e)
```

#### Function evaluation time

| length(x) | Python time (s) | Julia time (s) | Speed-Up vs. Python  |
|-----------|-----------------|----------------|----------------------|
| 16        | 5.60284e-5      | 2.74e-6        | 20.45                |
| 1600      | 0.00446105      | 4.1718e-5      | 106.93               |
| 16000     | 0.044596        | 0.000400089    | 111.47               |

#### Gradient evaluation time

| length(x) | AlgoPy time (s) | ForwardDiff time (s) | Speed-Up vs. AlgoPy |
|-----------|-----------------|----------------------|---------------------|
| 16        | 0.0019691       | 5.7223e-5            | 34.41               |
| 1600      | 0.549358        | 0.0328962            | 16.70               |
| 16000     | 89.5198         | 1.87736              | 47.68               |

#### Function-to-gradient slowdown ratio (lower is better)

$$\text{per-partial slowdown ratio} = \frac{\text{gradient time}}{\text{function time} \cdot \text{length}(\vec{x})}$$

| length(x) | AlgoPy Ratio  | ForwardDiff Ratio |
|-----------|---------------|-------------------|
| 16        | 2.20          | 1.31              |
| 1600      | 0.08          | 0.49              |
| 16000     | 0.13          | 0.29              |

#### vector-mode vs. chunk-mode (ForwardDiff only)

| length(x) | chunk size | chunk-mode time (s) | vector-mode time (s) | Speed-up vs. chunk-mode |
|-----------|------------|---------------------|----------------------|-------------------------|
| 16        | 1          | 5.6552e-5           | 5.7223e-5            | 0.99                    |
| 16        | 2          | 4.9823e-5           | 5.7223e-5            | 0.87                    |
| 16        | 4          | 4.6942e-5           | 5.7223e-5            | 0.82                    |
| 16        | 8          | 4.5825e-5           | 5.7223e-5            | 0.80                    |
| 16        | 16         | 4.4029e-5           | 5.7223e-5            | 0.78                    |
| 1600      | 1          | 0.171796            | 0.0328962            | 5.22                    |
| 1600      | 2          | 0.0966284           | 0.0328962            | 2.94                    |
| 1600      | 4          | 0.0679323           | 0.0328962            | 2.07                    |
| 1600      | 8          | 0.0484395           | 0.0328962            | 1.47                    |
| 1600      | 16         | 0.0420936           | 0.0328962            | 1.28                    |
| 16000     | 1          | 17.0998             | 1.87736              | 9.11                    |
| 16000     | 2          | 9.59618             | 1.87736              | 5.11                    |
| 16000     | 4          | 6.74976             | 1.87736              | 3.60                    |
| 16000     | 8          | 4.80788             | 1.87736              | 2.56                    |
| 16000     | 16         | 4.1689              | 1.87736              | 2.22                    |


#### C++ Comparison

| length(x) | function time (s)  | chunk size | chunk-mode time (s) | Speed-Up vs. ForwardDiff |
|-----------|--------------------|------------|---------------------|--------------------------|
| 16000     | 0.000397921        | 1          | 4.74544             | 3.60                     |


### Rosenbrock Function

#### Definition

$$f(\vec{x}) = \sum_{i=1}^{k-1} \big[(b - x_i)^2 + a \cdot (x_{i+1} - x_i^2)^2 \big]$$
#### Julia Code

```julia
sqr(i) = i*i

function rosenbrock(x)
    a, b = 100.0, 1.0
    result = zero(eltype(x))
    for i in 1:length(x)-1
        result += sqr(b - x[i]) + a*sqr(x[i+1] - sqr(x[i]))
    end
    return result
end
```

#### Python Code (with AlgoPy)

```python
def sqr(i):
    return i*i

def rosenbrock(x):
    a, b = 100.0, 1.0
    result = 0.0
    for i in xrange(len(x)-1):
        result += sqr(b - x[i]) + a*sqr(x[i+1] - sqr(x[i]))
    return result
```

#### Function evaluation time

| length(x) | Python time (s) | Julia time (s) | Speed-Up vs. Python  |
|-----------|-----------------|----------------|----------------------|
| 16        | 1.69277e-5      | 1.447e-6       | 11.70                |
| 1600      | 0.00180316      | 5.428e-6       | 332.20               |
| 16000     | 0.0162311       | 3.6963e-5      | 439.12               |

#### Gradient evaluation time

| length(x) | AlgoPy time (s) | ForwardDiff time (s) | Speed-Up vs. AlgoPy |
|-----------|-----------------|----------------------|---------------------|
| 16        | 0.00234795      | 5.8931e-5            | 39.84               |
| 1600      | 0.605024        | 0.035377             | 17.10               |
| 16000     | 98.9114         | 3.26842              | 30.26               |

#### Function-to-gradient slowdown ratio (lower is better)

$$\text{per-partial slowdown ratio} = \frac{\text{gradient time}}{\text{function time} \cdot \text{length}(\vec{x})}$$

| length(x) | AlgoPy Ratio  | ForwardDiff Ratio |
|-----------|---------------|-------------------|
| 16        | 8.70          | 2.55              |
| 1600      | 0.21          | 4.07              |
| 16000     | 0.38          | 5.53              |

#### vector-mode vs. chunk-mode (ForwardDiff only)

| length(x) | chunk size | chunk-mode time (s) | vector-mode time (s) | Speed-Up vs. chunk-mode |
|-----------|------------|---------------------|----------------------|-------------------------|
| 16        | 1          | 5.4143e-5           | 5.8931e-5            | 0.92                    |
| 16        | 2          | 4.4274e-5           | 5.8931e-5            | 0.75                    |
| 16        | 4          | 4.6301e-5           | 5.8931e-5            | 0.79                    |
| 16        | 8          | 4.7975e-5           | 5.8931e-5            | 0.81                    |
| 16        | 16         | 4.7291e-5           | 5.8931e-5            | 0.80                    |
| 1600      | 1          | 0.13801             | 0.035377             | 3.90                    |
| 1600      | 2          | 0.083086            | 0.035377             | 2.35                    |
| 1600      | 4          | 0.0727295           | 0.035377             | 2.06                    |
| 1600      | 8          | 0.0576288           | 0.035377             | 1.63                    |
| 1600      | 16         | 0.0619054           | 0.035377             | 1.75                    |
| 16000     | 1          | 13.7761             | 3.26842              | 4.21                    |
| 16000     | 2          | 8.28223             | 3.26842              | 2.53                    |
| 16000     | 4          | 7.26213             | 3.26842              | 2.22                    |
| 16000     | 8          | 5.74794             | 3.26842              | 1.76                    |
| 16000     | 16         | 6.20597             | 3.26842              | 1.90                    |

#### C++ Comparison

| length(x) | function time (s)  | chunk size | chunk-mode time (s) | Speed-Up vs. ForwardDiff |
|-----------|--------------------|------------|---------------------|--------------------------|
| 16000     | 3.50475e-5         | 1          | 0.751954            | 18.32                    |